In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [2]:
train_df = pd.read_csv('train.csv')
print train_df.columns

Index([u'id', u'qid1', u'qid2', u'question1', u'question2', u'is_duplicate'], dtype='object')


In [21]:
train_q1 = train_df['question1'].tolist()
train_q2 = train_df['question2'].tolist()
trainY = train_df['is_duplicate'].tolist()

In [22]:
print len(train_q1)
print len(train_q2)

del_idxs = []

for i,q in enumerate(train_q1):
    if type(q) != str:
        del_idxs.append(i)
print del_idxs
for i,q in enumerate(train_q2):
    if type(q) != str:
        del_idxs.append(i)
print del_idxs
del_idxs = list(set(del_idxs))

train_q1 = [train_q1[i] for i in range(len(train_q1)) if i not in del_idxs]    
train_q2 = [train_q2[i] for i in range(len(train_q2)) if i not in del_idxs]    
trainY = [trainY[i] for i in range(len(trainY)) if i not in del_idxs]    
print len(train_q1)
print len(train_q2)
print len(trainY)

404290
404290
[363362]
[363362, 105780, 201841]
404287
404287
404287


In [5]:
#nltk.download('stopwords')
tbl = dict.fromkeys(i for i in xrange(sys.maxunicode) if unicodedata.category(unichr(i)).startswith('P'))
stop_words = set(stopwords.words('english'))
def tokenize(text):
    #print text
    text = unicode(text)
    #text = re.sub(r'\d+', '', text)
    #print text
    tokens = nltk.word_tokenize(text)
    
    stems = []
    for item in tokens:
       if item not in stop_words:
            #stems.append(item)
           stems.append(PorterStemmer().stem(item))
    return stems

def preprocess(text):
    if type(text)==float:
        print text
    text = text.lower().translate(None, string.punctuation)
    return text

In [6]:
# Remove punctuation

train_q1 = [preprocess(q) for q in train_q1]
train_q2 = [preprocess(q) for q in train_q2]

In [7]:
# stemming and tokenize

wordCount = defaultdict(int)
for i,q in enumerate(train_q1):
    words = tokenize(q)
    for w in words:
        wordCount[w]+=1

In [8]:
# Limit the vocabulary
uni_sorted = sorted(wordCount.items(), key=operator.itemgetter(1)) [::-1]
vocab = [item[0] for item in uni_sorted[:10000]]
print vocab[:100]

[u'best', u'get', u'india', u'like', u'differ', u'use', u'peopl', u'good', u'way', u'would', u'make', u'one', u'quora', u'learn', u'time', u'life', u'money', u'know', u'work', u'mean', u'year', u'what', u'question', u'thing', u'indian', u'much', u'think', u'someon', u'new', u'start', u'go', u'engin', u'book', u'mani', u'becom', u'find', u'happen', u'trump', u'day', u'movi', u'ever', u'job', u'without', u'world', u'onlin', u'us', u'want', u'person', u'take', u'feel', u'better', u'girl', u'first', u'note', u'univers', u'love', u'compani', u'english', u'need', u'account', u'improv', u'possibl', u'program', u'2016', u'number', u'prepar', u'phone', u'compar', u'could', u'dont', u'buy', u'countri', u'chang', u'live', u'languag', u'ask', u'student', u'realli', u'weight', u'stop', u'app', u'lose', u'studi', u'500', u'websit', u'googl', u'1000', u'answer', u'busi', u'donald', u'import', u'long', u'develop', u'effect', u'war', u'sex', u'colleg', u'friend', u'presid', u'look']


In [9]:
# Create a single list of questions
train = train_q1+train_q2
vectorizer = TfidfVectorizer(tokenizer=tokenize, vocabulary=vocab)
train_tfidf = vectorizer.fit_transform(train)

In [23]:
num_train = 390000

trainX = sparse.hstack([train_tfidf[:num_train], train_tfidf[len(train_q1):len(train_q1)+num_train]])
valX = sparse.hstack([train_tfidf[num_train:len(train_q1)], train_tfidf[len(train_q1) +num_train:]])

trainY = np.array(trainY)
valY = trainY[num_train:]
trainY = trainY[:num_train]
print train_tfidf.shape
print len(train_q1)
print len(train_q2)
print len(train)
print trainX.shape
print valY.shape

(808574, 10000)
404287
404287
808574
(390000, 20000)
(14287,)


In [24]:
xgb1 = xgb.XGBClassifier()
xgb1.fit(trainX, trainY)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [25]:
preds_train = xgb1.predict(trainX)
preds_val = xgb1.predict(valX)

In [26]:
acc_train = np.mean(trainY==preds_train)
acc_val = np.mean(valY==preds_val)

print 'Training accuracy: ', acc_train
print 'Validation accuracy: ', acc_val

Training accuracy:  0.691535897436
Validation accuracy:  0.713095821376


In [ ]:
print 

In [28]:
average_precision = average_precision_score(valY, preds_val)

print('Average precision-recall score on validation data: {0:0.2f}'.format(
      average_precision))


Average precision-recall score on validation data: 0.44


In [31]:
precision, recall, _ = precision_recall_curve(valY, preds_val)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.savefig('roc.PNG')